# Import Modules

In [1]:
import h2o
import pandas

In [2]:
project_path = "/gtc-2017"

# Connect or Start H2O

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,53 mins 23 secs
H2O cluster version:,3.11.0.230
H2O cluster version age:,14 hours and 39 minutes
H2O cluster name:,root
H2O cluster total nodes:,1
H2O cluster free memory:,25.76 Gb
H2O cluster total cores:,20
H2O cluster allowed cores:,20
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


# Import Data

Import data into H2O.

In [4]:
mnist_training = h2o.import_file(project_path+"/data/mnist-training.csv")
mnist_testing = h2o.import_file(project_path+"/data/mnist-testing.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Change the "label" column to a factor (i.e. categorical feature).

In [5]:
mnist_training["label"] = mnist_training["label"].asfactor()
mnist_testing["label"] = mnist_testing["label"].asfactor()

# Explore Data

In [6]:
mnist_training.head()

uri,label
/gtc-2017/data/mnist_png/training/6/6453.png,6
/gtc-2017/data/mnist_png/training/6/13970.png,6
/gtc-2017/data/mnist_png/training/6/21967.png,6
/gtc-2017/data/mnist_png/training/6/11277.png,6
/gtc-2017/data/mnist_png/training/6/27350.png,6
/gtc-2017/data/mnist_png/training/6/32119.png,6
/gtc-2017/data/mnist_png/training/6/44201.png,6
/gtc-2017/data/mnist_png/training/6/58770.png,6
/gtc-2017/data/mnist_png/training/6/39440.png,6
/gtc-2017/data/mnist_png/training/6/41988.png,6


Get the number of output classes from the data.

In [7]:
num_classes = mnist_training["label"].nlevels()[0]
print(num_classes)

10


# Test GPU Driver

In [8]:
!nvidia-smi

Tue May  9 04:12:49 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 378.13                 Driver Version: 378.13                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 0000:02:00.0      On |                  N/A |
| 31%   40C    P8    11W / 180W |   4785MiB /  8112MiB |      8%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Define Custom Network

In [9]:
def simple_model(w, h, channels, num_classes):
    import json
    import tensorflow as tf
    from tensorflow.python.framework import ops
    
    graph = tf.Graph()
    with graph.as_default():
        size = w * h * channels
        x = tf.placeholder(tf.float32, [None, size])
        W = tf.Variable(tf.zeros([size, num_classes]))
        b = tf.Variable(tf.zeros([num_classes]))
        y = tf.matmul(x, W) + b
        
        predictions = tf.nn.softmax(y)
        
        y_ = tf.placeholder(tf.float32, [None, num_classes])
        
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
        
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
        
        tf.add_to_collection(ops.GraphKeys.TRAIN_OP, train_step)
        tf.add_to_collection("predictions", predictions)
        
        global_step = tf.Variable(0, name="global_step", trainable=False)
        
        init = tf.global_variables_initializer()
        tf.add_to_collection(ops.GraphKeys.INIT_OP, init.name)
        tf.add_to_collection("logits", y)
        saver = tf.train.Saver()
        meta = json.dumps({
            "inputs": {"batch_image_input": x.name, "categorical_labels": y_.name},
            "outputs": {"categorical_logits": y.name},
            "parameters": {"global_step": global_step.name}
        })
        
        tf.add_to_collection("meta", meta)
        filename = project_path+"/models/mymodel_tensorflow.meta"
        tf.train.export_meta_graph(filename, saver_def=saver.as_saver_def())
    return(filename)

Instantiate an instance of the network.

In [10]:
network_def_path = simple_model(28, 28, 1, num_classes)

# Build Deep Water TensorFlow Model

In [11]:
from h2o.estimators.deepwater import H2ODeepWaterEstimator
model_mnist_mymodel_tf = H2ODeepWaterEstimator(epochs=80,
                                            network_definition_file=network_def_path,
                                            backend="tensorflow",
                                            image_shape=[28,28],
                                            channels=1, model_id="model_mnist_mymodel_tf")
model_mnist_mymodel_tf.train(x=["uri"], y="label", training_frame=mnist_training, validation_frame=mnist_testing)

deepwater Model Build progress: |█████████████████████████████████████████| 100%


In [12]:
model_mnist_mymodel_tf.show()

Model Details
H2ODeepWaterEstimator :  Deep Water
Model Key:  model_mnist_mymodel_tf


ModelMetricsMultinomial: deepwater
** Reported on train data. **

MSE: 0.08646877789739044
RMSE: 0.29405573943963487
LogLoss: 2.984679138655652
Mean Per-Class Error: 0.08760483306433123
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
957.0,0.0,7.0,2.0,1.0,17.0,3.0,5.0,11.0,1.0,0.0468127,"47 / 1,004"
0.0,1099.0,19.0,1.0,1.0,0.0,1.0,7.0,6.0,1.0,0.0317181,"36 / 1,135"
8.0,7.0,944.0,4.0,2.0,5.0,5.0,13.0,23.0,6.0,0.0717797,"73 / 1,017"
1.0,6.0,52.0,864.0,2.0,49.0,4.0,12.0,34.0,12.0,0.1660232,"172 / 1,036"
0.0,2.0,9.0,0.0,880.0,5.0,5.0,25.0,7.0,56.0,0.1102123,109 / 989
6.0,3.0,12.0,35.0,3.0,785.0,10.0,4.0,26.0,9.0,0.1209406,108 / 893
7.0,1.0,11.0,0.0,3.0,16.0,946.0,4.0,6.0,2.0,0.0502008,50 / 996
1.0,3.0,7.0,3.0,6.0,1.0,0.0,990.0,1.0,29.0,0.0489914,"51 / 1,041"
3.0,9.0,28.0,22.0,5.0,30.0,3.0,5.0,862.0,13.0,0.1204082,118 / 980
10.0,1.0,2.0,17.0,12.0,11.0,0.0,46.0,8.0,875.0,0.1089613,107 / 982


Top-10 Hit Ratios: 


k,hit_ratio
1,0.9135312
2,0.919587
3,0.919587
4,0.919587
5,0.919587
6,0.919587
7,0.919587
8,0.919587
9,0.919587
10,1.0



ModelMetricsMultinomial: deepwater
** Reported on validation data. **

MSE: 0.0942
RMSE: 0.3069201850644561
LogLoss: 3.253552736400621
Mean Per-Class Error: 0.09549782192578034
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
956.0,0.0,0.0,3.0,0.0,11.0,3.0,3.0,2.0,2.0,0.0244898,24 / 980
0.0,1109.0,13.0,0.0,0.0,1.0,4.0,1.0,7.0,0.0,0.0229075,"26 / 1,135"
7.0,7.0,937.0,8.0,5.0,7.0,15.0,9.0,32.0,5.0,0.0920543,"95 / 1,032"
4.0,2.0,52.0,838.0,2.0,56.0,2.0,13.0,30.0,11.0,0.1702970,"172 / 1,010"
2.0,3.0,13.0,1.0,859.0,2.0,12.0,17.0,13.0,60.0,0.1252546,123 / 982
11.0,4.0,3.0,31.0,7.0,770.0,14.0,7.0,40.0,5.0,0.1367713,122 / 892
12.0,1.0,14.0,1.0,4.0,29.0,889.0,1.0,7.0,0.0,0.0720251,69 / 958
2.0,6.0,24.0,6.0,5.0,2.0,0.0,952.0,2.0,29.0,0.0739300,"76 / 1,028"
8.0,8.0,15.0,12.0,5.0,40.0,9.0,12.0,852.0,13.0,0.1252567,122 / 974
11.0,9.0,0.0,6.0,16.0,8.0,1.0,49.0,13.0,896.0,0.1119921,"113 / 1,009"


Top-10 Hit Ratios: 


k,hit_ratio
1,0.9058
2,0.9129
3,0.9129
4,0.9129
5,0.9129
6,0.9129
7,0.9129
8,0.9129
9,0.9129
10,1.0


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2017-05-09 04:12:51,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan
,2017-05-09 04:12:51,0.838 sec,10138 obs/sec,0.0170667,1,1024.0,0.5317293,9.7624729,0.2827360,0.5208647,9.3703700,0.2713
,2017-05-09 04:12:58,7.586 sec,47180 obs/sec,5.0688,297,304128.0,0.3203914,3.5454279,0.1026507,0.3200000,3.5361700,0.1024
,2017-05-09 04:13:03,12.565 sec,45813 obs/sec,8.448,495,506880.0,0.3777017,4.9272532,0.1426586,0.3764306,4.8941446,0.1417
,2017-05-09 04:13:08,17.595 sec,43663 obs/sec,11.4346667,670,686080.0,0.4117793,5.8564708,0.1695622,0.4153312,5.9561915,0.1725
,2017-05-09 04:13:13,22.626 sec,42470 obs/sec,14.4384,846,866304.0,0.3934084,5.3455726,0.1547702,0.4093898,5.7886989,0.1676
,2017-05-09 04:13:18,27.694 sec,42091 obs/sec,17.5957333,1031,1055744.0,0.3109254,3.3349102,0.0966941,0.3117691,3.3571691,0.0972
,2017-05-09 04:13:23,32.721 sec,41814 obs/sec,20.7189333,1214,1243136.0,0.3160236,3.4485574,0.0998709,0.3204684,3.5471323,0.1027
,2017-05-09 04:13:28,37.734 sec,41558 obs/sec,23.808,1395,1428480.0,0.3259211,3.6688663,0.1062246,0.3388215,3.9650515,0.1148
,2017-05-09 04:13:33,42.806 sec,41392 obs/sec,26.9482667,1579,1616896.0,0.3112759,3.3465547,0.0968927,0.3264966,3.6818336,0.1066
